In [9]:
import pandas as pd

# Load your real dataset
df = pd.read_csv("../data/processed/cleaned_data.csv")

# Create the table for embeddings
df_to_embed = pd.DataFrame({
    "listing_id": range(len(df)),  # Generate unique ID from index
    "title": df["title"],
    "subtitle": df["subtitle"],
    "pred_price": df["buy_price"],  # Use actual price
    "shap_top3": "[]"               # placeholder for now
})

# Save it
df_to_embed.to_csv("../data/processed/listings_with_preds.csv", index=False)
print("Created listings_with_preds.csv with", len(df_to_embed), "rows")


Created listings_with_preds.csv with 21707 rows


In [10]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np
for col in ["neighborhood_id", "house_type_id"]:
    df[col] = df[col].astype("category").cat.codes

# Define features and target
features = [
    "sq_mt_built", "n_rooms", "n_bathrooms", "neighborhood_id", 
    "house_type_id", "has_terrace", "has_lift", "is_exterior",
    "log_sq_mt_built", "building_age"
]
target = "log_buy_price"

# Prepare data
df_model = df[features + [target]].dropna()
X = df_model[features]
y = df_model[target]

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost regressor
model = xgb.XGBRegressor(
    n_estimators=200,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5
print("XGBoost RMSE:", round(rmse, 3))

# Predict on full data and save
df["pred_price"] = np.expm1(model.predict(df[features].fillna(0)))



XGBoost RMSE: 0.222


In [ ]:
import shap
import json

# 1. Create SHAP explainer
explainer = shap.Explainer(model)
shap_values = explainer(X)

# 2. Get top-3 features per listing (by absolute SHAP value)
top3_idxs = np.argsort(-np.abs(shap_values.values), axis=1)[:, :3]
top3_features = [[features[i] for i in row] for row in top3_idxs]

# 3. Reload the listings_with_preds file
df_preds = pd.read_csv("../data/processed/listings_with_preds.csv")

# 4. Add SHAP top 3 as a stringified JSON list
df_preds["shap_top3"] = [json.dumps(lst) for lst in top3_features]

# 5. Save it again
df_preds.to_csv("../data/processed/listings_with_preds.csv", index=False)
print("SHAP explanations added and saved")



[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for shap from https://files.pythonhosted.org/packages/e5/bb/dc75933de86e6076f58cf68325877be952a97a371c26b252013f1258a5a7/shap-0.47.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for slicer==0.0.8 from https://files.pythonhosted.org/packages/63/81/9ef641ff4e12cbcca30e54e72fb0951a2ba195d0cda0ba4100e532d929db/slicer-0.0.8-py3-none-any.whl.metadata
  Obtaining dependency information for numba>=0.54 from https://files.pythonhosted.org/packages/0f/a4/2b309a6a9f6d4d8cfba583401c7c2f9ff887adb5d54d8e2e130274c0973f/numba-0.61.2-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for cloudpickle from https://files.pythonhosted.org/packages/7e/e8/64c37fadfc2816a7701fa8a6ed8d87327c7d54eacfbfb6edab14a2f2be75/cloudpickle-3.1.1-py3-none-any.whl.metadata
  Using cached cloudpickle-3.1.1-py3-none-any.whl.metadata (7.1 kB)
  Obtaining dependency information for llvmlite<0.45,>=0.44.0dev0 from https://files.pythonhosted.org/packa

In [17]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = api_key


In [18]:
from sentence_transformers import SentenceTransformer
import pandas as pd, json, numpy as np, pathlib

df = pd.read_csv("../data/processed/listings_with_preds.csv")
model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

texts, ids = [], []
for _, r in df.iterrows():
    shap_list = json.loads(r["shap_top3"])  # even if empty
    txt = (
        f"Title: {r['title']}\n"
        f"Neighbourhood: {r['subtitle']}\n"
        f"Top-3 price drivers: {', '.join(shap_list)}\n"
        f"Predicted price: €{r['pred_price']:,.0f}"
    )
    texts.append(txt)
    ids.append(int(r["listing_id"]))

emb = model.encode(texts, convert_to_numpy=True).astype("float32")

pathlib.Path("vectorstore").mkdir(exist_ok=True)
np.save("vectorstore/embeddings.npy", emb)
np.save("vectorstore/ids.npy", np.array(ids))

print("Saved", emb.shape[0], "embeddings to vectorstore/")


Saved 21707 embeddings to vectorstore/


In [20]:
# import numpy as np
# import pathlib

# texts = []
# ids   = []

# for _, r in df.iterrows():
#     shap_list = json.loads(r["shap_top3"])
#     txt = (
#         f"Title: {r['title']}\n"
#         f"Neighbourhood: {r['subtitle']}\n"
#         f"Top-3 price drivers: {', '.join(shap_list)}\n"
#         f"Predicted price: €{r['pred_price']:,.0f}"
#     )
#     texts.append(txt)
#     ids.append(int(r["listing_id"]))

# emb_matrix = model.encode(texts, convert_to_numpy=True).astype("float32")   # shape (N, 384)

# # Save
# pathlib.Path("vectorstore").mkdir(exist_ok=True)
# np.save("vectorstore/embeddings.npy", emb_matrix)
# np.save("vectorstore/ids.npy", np.array(ids))

# print("Saved", emb_matrix.shape[0], "embeddings → vectorstore/")


In [21]:
import faiss
import numpy as np

# Load previously saved embeddings and index
emb = np.load("vectorstore/embeddings.npy")
ids = np.load("vectorstore/ids.npy")
index = faiss.read_index("vectorstore/madrid.faiss")


In [22]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

query = "modern flat with terrace in chamartín"
q_vec = model.encode(query, convert_to_numpy=True).astype("float32")
faiss.normalize_L2(q_vec.reshape(1, -1))

D, I = index.search(q_vec.reshape(1, -1), k=5)
print("Top 5 matching listing IDs:", ids[I[0]])


Top 5 matching listing IDs: [14277 15712 14856 15021 14863]


In [23]:
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
import pandas as pd

# 1. Load your data
df = pd.read_csv("../data/processed/listings_with_preds.csv")

docs = [
    Document(
        page_content=f"Listing {row.listing_id}: {row.title} in {row.subtitle}, price €{row.pred_price}",
        metadata={"listing_id": int(row.listing_id)}
    )
    for _, row in df.iterrows()
]

# 2. Use local model to avoid OpenAI embedding mismatch
hf_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

# 3. Build the new vectorstore from scratch
vectorstore = FAISS.from_documents(docs, embedding)
retriever = vectorstore.as_retriever()


C:\Users\Leon\AppData\Local\Temp\ipykernel_11988\3209062961.py:22: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")


In [24]:
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

response = qa.invoke("Why are homes in Chamartín more expensive?")
print("🤖", response["result"])


C:\Users\Leon\AppData\Local\Temp\ipykernel_11988\184841102.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)


🤖 Homes in Chamartín may be more expensive due to factors such as the neighborhood's location, amenities, infrastructure, demand, and overall desirability. Chamartín is a well-established and sought-after neighborhood in Madrid, known for its upscale properties, proximity to the city center, good transportation connections, green spaces, and quality of life. These factors can contribute to higher property prices compared to other areas.
